In [1]:
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import keras
from sklearn.dummy import DummyClassifier
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from numpy import array
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from pandas.plotting import lag_plot
from sklearn.feature_selection import SelectKBest, chi2

Using TensorFlow backend.


In [2]:
#Function for loading data!

def load_data():
    
    df = pickle.load(open('df_down_sampled.p','rb'))
    df = df.drop(['resp_1', 'resp_2','resp_3','resp_4'], axis = 1)
    df = df.sort_values(by = 'ts_id')
    df['y'] = 0
    mask = df.resp > 0
    df.loc[mask,'y'] = 1    
    
    return df

In [3]:
#Function for performing pca-transformation!

def perform_pca_transformation(X_train, X_test, number_of_components):
    
    col_list = []

    for x in range(number_of_components):
        
        col_list.append('pca_feature_' + str(x))
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    pca = PCA(n_components = number_of_components)
    pca.fit(X_train_scaled)
    
    X_train_pca = pca.transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    X_train_pca = pd.DataFrame(data = X_train_pca, columns = col_list)
    X_test_pca = pd.DataFrame(data = X_test_pca, columns = col_list)
    
    return (X_train_pca, X_test_pca)

In [4]:
#Function for splitting data into train/test set!

def train_test_split(test_share, data):
    
    #Split data into initial train/test
    
    train_share = 1 - test_share    
    train_size = int(len(data) * train_share)
    train_set = data[0:train_size]
    test_set = data[train_size:len(data)]    
    
    
    return (train_set, test_set)

In [5]:
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        
        col_type = df[col].dtype
        
        if col_type != 'object':
           
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:                    
                    df[col] = df[col].astype(np.int8)
                
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [6]:
#Load data!

df = load_data()

In [8]:
df = df[df.weight > 0]

In [9]:
len(df)

433544

In [10]:
#Split into train/test!

train_set, test_set = train_test_split(test_share = 0.3, data = df)

In [11]:
train_set.shape

(303480, 135)

In [12]:
test_set.shape

(130064, 135)

In [13]:
train_set = reduce_memory_usage(train_set)

Memory usage of dataframe is 314.89013671875 MB


C:\Users\mathias.buxhoeveden\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\mathias.buxhoeveden\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\mathias.buxhoeveden\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Memory usage of dataframe after reduction 80.45906066894531 MB
Reduced by 74.44852941176471 % 


C:\Users\mathias.buxhoeveden\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
X_train = train_set.drop(['date','weight','y','ts_id', 'resp'], axis = 1)
X_test = test_set.drop(['date','weight','y','ts_id', 'resp'], axis = 1)

y_train = train_set.y
y_test = test_set.y

In [27]:
model = XGBClassifier(n_trees = 1, max_deepth = 2) 
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.300000012, max_deepth=2, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=nan,
       monotone_constraints=None, n_estimators=100, n_jobs=0, n_trees=1,
       num_parallel_tree=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
       tree_method=None, validate_parameters=False, verbosity=None)

In [28]:
y_pred_train = model.predict_proba(X_train)[:,1]

In [29]:
train_roc = roc_auc_score(y_train, y_pred_train)

In [30]:
y_pred_test = model.predict_proba(X_test)[:,1]

In [31]:
test_roc = roc_auc_score(y_test, y_pred_test)

In [32]:
print("Train roc: {}, test roc: {}".format(train_roc, test_roc))

Train roc: 0.7402494541691331, test roc: 0.5226091336724686


In [20]:
print("Train roc: {}, test roc: {}".format(train_roc, test_roc))

Train roc: 0.7402494541691331, test roc: 0.5226091336724686
